In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
# make your Google Drive accessible
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/course-v3/nbs/dl2/'

# navigate to the notebooks directory for dl2
import os
os.chdir(base_dir)

# update fastai, according to instructions at https://course.fast.ai/start_colab.html
!curl -s https://course.fast.ai/setup/colab | bash
    
# install fire
!pip install fire
!pip install nbconvert

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Updating fastai...
Done.
     |████████████████████████████████| 81kB 3.1MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=d00eb5b70253c21c4b1acba844a6954cda6a2cfa8c174e513df733f4361f98ca
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built fire


## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [0]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

Mean is not 0 and std is not 1

In [5]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

Hence we normalize them

In [0]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

Note that we do not normalize the validation set with its own mean and std because then the training and the validation sets will be on different scales

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.0001), tensor(1.))

In [0]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [0]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_train.max() + 1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

Just to simplify things we will use mse as our error function instead of cross entropy and 1 activation as our output instead of 10

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
nh = 50

In [0]:
w1 = torch.randn(m, nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)
b2 = torch.randn(1)

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

Numbers with mean 0 and std 1 / root(inp) will give an output that has mean 0 and std 1. Write about fixup initialization in article

In [13]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [0]:
def lin(x, w, b): return x@w + b

In [0]:
t = lin(x_valid, w1, b1)

In [16]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(1.1935), tensor(27.7240))

In [0]:
# simplified version of kaiming init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [0]:
t = lin(x_valid, w1, b1)

In [19]:
t.mean(),t.std()

(tensor(0.0565), tensor(1.0197))

In [0]:
# after kaiming init
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [0]:
def relu(x): return x.clamp_min(0.)

In [0]:
t = relu(lin(x_valid, w1, b1))

In [23]:
t.mean(), t.std()

(tensor(0.4314), tensor(0.6375))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [0]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)

In [0]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [26]:
w1.mean(),w1.std()

(tensor(-0.0004), tensor(0.0504))

In [27]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5452), tensor(0.8337))

In [0]:
#export
from torch.nn import init

Fan out means do you want to preserve the variance in the forward or the backward pass. If we divide by m i.e. input, it'll preserve in the forward pass and if we divide by nh i.e. output it'll preserve in the backward pass

In [0]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [0]:
init.kaiming_normal_??

In [31]:
w1.mean(),w1.std()

(tensor(-0.0003), tensor(0.0503))

In [32]:
t.mean(),t.std()

(tensor(0.5252), tensor(0.7879))

In [33]:
w1.shape

torch.Size([784, 50])

In [0]:
import torch.nn

In [35]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [0]:
torch.nn.Linear.forward??

In [0]:
torch.nn.functional.linear??

In [0]:
torch.nn.Conv2d??

In [0]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [0]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [41]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0149), tensor(0.7597))

In [0]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [43]:
t3 = model(x_valid)
t3.mean(), t3.std()

(tensor(0.1807), tensor(0.6239))

In [44]:
%timeit -n 10 _=model(x_valid)

10 loops, best of 3: 20.7 ms per loop


In [0]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [46]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [0]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [0]:
y_train, y_valid = y_train.float(), y_valid.float()

In [0]:
preds = model(x_train)

In [50]:
preds.shape

torch.Size([50000, 1])

In [51]:
mse(preds, y_train)

tensor(26.5406)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [0]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [0]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [0]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [0]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [0]:
forward_and_backward(x_train, y_train)

In [0]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [0]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [0]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [0]:
loss = forward(xt2, y_train)

In [0]:
loss.backward()

In [0]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [0]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [0]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [0]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [0]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [68]:
%time loss = model(x_train, y_train)

CPU times: user 121 ms, sys: 0 ns, total: 121 ms
Wall time: 126 ms


In [69]:
%time model.backward()

CPU times: user 3.46 s, sys: 3.63 s, total: 7.08 s
Wall time: 7.15 s


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [0]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [0]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [0]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [0]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [77]:
%time loss = model(x_train, y_train)

CPU times: user 117 ms, sys: 1.56 ms, total: 118 ms
Wall time: 119 ms


In [78]:
%time model.backward()

CPU times: user 245 ms, sys: 16.2 ms, total: 262 ms
Wall time: 283 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [0]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [0]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [82]:
%time loss = model(x_train, y_train)

CPU times: user 109 ms, sys: 0 ns, total: 109 ms
Wall time: 110 ms


In [83]:
%time model.backward()

CPU times: user 235 ms, sys: 6.88 ms, total: 242 ms
Wall time: 247 ms


In [0]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [0]:
#export
from torch import nn

In [0]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [0]:
model = Model(m, nh, 1)

In [88]:
%time loss = model(x_train, y_train)

CPU times: user 110 ms, sys: 311 µs, total: 110 ms
Wall time: 114 ms


In [89]:
%time loss.backward()

CPU times: user 95.8 ms, sys: 0 ns, total: 95.8 ms
Wall time: 95 ms


## Export

In [90]:
!./notebook2script.py 02_fully_connected.ipynb

/bin/bash: ./notebook2script.py: /usr/bin/env: bad interpreter: Permission denied
